In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import io
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from preprocessing import flip, color, rotate, zoom

In [ ]:
valid_data = tfds.load('cats_vs_dogs', split='train[:10%]')

train_data, info = tfds.load('cats_vs_dogs', split='train[10%:100%]', with_info=True)

test_data = tfds.load('cats_vs_dogs', split=tfds.Split.TEST)

In [ ]:
print(info.features['label'].names)
decoder = info.features['label'].names

In [ ]:
for data in train_data.take(1):
    img = data['image']
    label = data['label']

plt.title(decoder[label])
plt.imshow(img)

In [ ]:
input_shape = (299, 299)

def parse_aug_fn(dataset):
    x = tf.cast(dataset['image'], tf.float32) / 255.
    x = tf.image.resize(x, input_shape)
    
    x = flip(x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: color(x), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.75, lambda: rotate(x), lambda: x)
    x = tf.cond(tf.random.uniform([], 0, 1) > 0.5, lambda: zoom(x), lambda: x)
    
    return x, dataset['label']

def parse_fn(dataset):
    x = tf.cast(dataset['image'], tf.float32) / 255.
    x = tf.image.resize(x, input_shape)
    
    return x, dataset['label']

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
buffer_size = 1000
batch_size = 64

train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.shuffle(train_num)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

valid_data = valid_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

## Model 1

In [ ]:
model_dir = 'lab9-logs/models'
os.makedirs(model_dir)

log_dir = os.path.join('lab9-logs', 'model-1')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_esp = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=30, mode='max')

base_model = tf.keras.applications.InceptionsV3(include_top=False, weights=None, pooling='avg', input_shape=input_shape+(3,))

model_1 = tf.keras.Sequential([base_model, 
                               layers.Dense(128, activation='relu'), 
                               layers.Dense(1, activation='sigmoid')])

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics=[keras.metrics.BinaryAccuracy()])

history = model_1.fit(train_data, epochs=200, validation_data=valid_data, callbacks=[model_cbk, model_esp])

## Model 2

In [ ]:
log_dir = os.path.join('lab9-logs', 'model-2')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_esp = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=30, mode='max')

module_url = 'https://tfhub.dev/google.tf2-preview/inception_v3/feature_vector/2'

model_2 = tf.keras.Sequential([hub.KerasLayer(module_url,
                                              input_shape=(299, 299, 3),
                                              output_shape=(2048,),
                                              trainable=False),
                               layers.Dense(128, activation='relu'),
                               layers.Dense(1, activation='sigmoid')])

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics=[keras.metrics.BinaryAccuracy()])

history = model_2.fit(train_data, epochs=200, validation_data=valid_data, callbacks=[model_cbk, model_esp])